<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/ProcDados_dask_assignment_erike_simon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CESAR School Recife**

**Disciplina:** *Processamento de Dados em Larga Escala*

**Alunos:** *Erike Simon, José Aparecido*

## Sobre os dados

O arquivo CSV contém eventos 'click' ou 'view' no tempo, de usuários em anúncios de determinadas campanhas.

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Nome do arquivo CSV:**  
data/ad_action.csv

## Sobre as questões

As questões devem ser respondidas usando a tecnologia Dask.

Quando utilizar pandas tenha cuidado para evitar estouro de memória, sempre imaginado que vai executar o código com uma grande massa de dados.

Mesmo que não consiga terminar alguma questão, favor enviar, porque parte do código pode valer alguma pontuação.

## Configurações, imports e instalações

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade "dask[dataframe]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.5 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2023.8.1
    Uninstalling dask-2023.8.1:
      Successfully uninstalled dask-2023.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.8.1 requires dask==2023.8.1, but you have dask 2024.1.1 which is incompatible.


In [ ]:
!pip install --upgrade "dask[distributed]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
  Attempting uninstall: distributed
    Found existing installation: distributed 2023.8.1
    Uninstalling distributed-2023.8.1:
      Successfully uninstalled distributed-2023.8.1


In [ ]:
!pip install --upgrade pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:39699
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34177'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45381'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45803', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45803
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:45198
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42873', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker co

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 12.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39699,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 12.67 GiB
Comm: tcp://127.0.0.1:42873,Total threads: 1
Dashboard: http://127.0.0.1:37591/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:34177,


In [ ]:
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/proc-dados-larga-escala/data/ad_action.csv', header=None, parse_dates=[0])
df.columns = ['timestamp','user_id','action','adId','campaignId']
df.head()

,timestamp,user_id,action,adId,campaignId
0,2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01
1,2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02
2,2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02
3,2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03
4,2016-01-02 04:57:00,fef9a98c-d73e-48ef-b2cb-766ba85dc3e3,view,adId_02,campaignId_02


In [ ]:
df.dtypes

timestamp     datetime64[ns]
user_id               object
action                object
adId                  object
campaignId            object
dtype: object

## 1) Quais são as top 3 campanhas que geraram mais eventos? Ordene pela quantidade de eventos (2,5 pontos)

In [ ]:
# Quantas campanhas temos?

unique_campaigns = df['campaignId'].unique().compute()

print(unique_campaigns)

0    campaignId_01
1    campaignId_02
2    campaignId_03
Name: campaignId, dtype: object


In [ ]:
# Quantos anúncios temos?

unique_ads = df['adId'].unique().compute()

print(unique_ads)

0    adId_09
1    adId_02
2    adId_07
3    adId_05
4    adId_03
5    adId_06
6    adId_08
7    adId_04
8    adId_10
9    adId_01
Name: adId, dtype: object


In [ ]:
most_popular_campaigns = df.groupby('campaignId')\
    ['timestamp']\
    .count()\
    .rename('event_count')\
    .compute()\

most_popular_campaigns.sort_values(ascending=False)

campaignId
campaignId_02    91216
campaignId_03    87036
campaignId_01    76461
Name: event_count, dtype: int64

**Resposta:**

1. Agrupamos por 'campaignId';
3. A contagem dos eventos foi feita por 'timestamp';
4. Ordenamos os valores.

Resultado:
* campaignId_02    91216
* campaignId_03    87036
* campaignId_01    76461

Sendo a campanha 2 a mais popular entre as 3.


## 2) Qual campanha teve mais clicks? (2,5 pontos)

In [ ]:
# Observando a quantidade de clicks
campaign_clicks = df[df['action'] == 'click']\
        .groupby('campaignId')\
        ['timestamp']\
        .count()\
        .rename('click_count')\
        .compute()

campaign_clicks.sort_values(ascending=False)

campaignId
campaignId_02    63983
campaignId_03    60947
campaignId_01    53375
Name: click_count, dtype: int64

In [ ]:
# Observando a quantidade de views

campaign_views = df[df['action'] == 'view']\
        .groupby('campaignId')\
        ['timestamp']\
        .count()\
        .rename('view_count')\
        .compute()

campaign_views.sort_values(ascending=False)

campaignId
campaignId_02    27233
campaignId_03    26089
campaignId_01    23086
Name: view_count, dtype: int64

**Resposta:**

1. Filtramos as linhas que possuem 'action' = 'click';
2. Agrupamos por 'campaignId';
3. A contagem dos eventos foi feita por 'timestamp'
4. Ordenamos os valores pela campanha que teve mais clicks.

A **campanha 2** teve mais clicks em relação as demais. Foram **63983 clicks**. Observa-se também que a campanha 2 teve a maior quantidade de *views*. Um dos motivos de obter maior quantidade de *clicks* e *views* seja por ser a campanha mais popular, como observado na resposta 1.

## 3) Qual mês teve o maior total de eventos acumulado? (2,5 pontos)

In [ ]:
# Observando o período mínimo e máximo dos dados

periodo_minimo = df['timestamp'].min().compute()
periodo_maximo = df['timestamp'].max().compute()

print(f"Período Mínimo: {periodo_minimo}")
print(f"Período Máximo: {periodo_maximo}")

Período Mínimo: 2016-01-01 00:00:00
Período Máximo: 2016-12-29 23:47:00


In [ ]:
# Agrupando por mês e calculando o total de eventos

month_events = df.groupby(df['timestamp'].dt.to_period("M"))\
      .size()\
      .reset_index()\
      .compute()

month_events.columns = ['mounth', 'accu_events']

month_events

,mounth,accu_events
0,2016-01,25800
1,2016-02,19619
2,2016-03,21377
3,2016-04,20558
4,2016-05,21224
5,2016-06,20657
6,2016-07,21183
7,2016-08,21362
8,2016-09,20627
9,2016-10,21363


In [ ]:
month_events.nlargest(n=1, columns='accu_events')

,mounth,accu_events
0,2016-01,25800


**Resposta:**

1. Agrupamos por 'timestamp', extraindo a informação do mês;
2. 'size()' para calcular a quantidade de eventos em cada mês;
3. 'reset_index()' para converter o resultado de volta para um DataFrame;
4. Renomeamos as colunas;
5. 'nlasgest()' para obter o mês com mais eventos acumulado.

O **mês 1** do ano de 2016 teve mais eventos acumulados.

## 4) Nas situações onde existe um evento de view seguido de um evento de click criados pelo mesmo usuário no mesmo anúncio e campanha, quais são os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos (2,5 pontos)

In [ ]:
# Ordenando os dados por 'user_id' e 'timestamp'
df_sorted = df.sort_values(by=['user_id', 'timestamp']).compute()
df_sorted

,timestamp,user_id,action,adId,campaignId
154120,2016-01-13 21:09:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01
247604,2016-01-14 09:30:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01
175723,2016-03-18 11:24:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01
200627,2016-03-18 18:25:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01
214537,2016-04-02 19:42:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01
...,...,...,...,...,...
22729,2016-12-15 18:09:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01
74806,2016-12-15 18:37:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01
31330,2016-12-15 19:05:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,view,adId_10,campaignId_01
124805,2016-12-18 05:58:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01


In [ ]:
# Criação uma coluna 'next_action' para a próxima ação e uma 'prev_action' para a ação anterior
df_sorted['next_action'] = df_sorted.groupby('user_id')['action'].shift(-1)
df_sorted['prev_action'] = df_sorted.groupby('user_id')['action'].shift(1)

df_sorted

,timestamp,user_id,action,adId,campaignId,next_action,prev_action
154120,2016-01-13 21:09:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,NaN
247604,2016-01-14 09:30:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,click
175723,2016-03-18 11:24:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,click
200627,2016-03-18 18:25:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,click
214537,2016-04-02 19:42:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,view,click
...,...,...,...,...,...,...,...
22729,2016-12-15 18:09:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01,click,view
74806,2016-12-15 18:37:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01,view,click
31330,2016-12-15 19:05:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,view,adId_10,campaignId_01,click,click
124805,2016-12-18 05:58:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01,view,view


In [ ]:
# Filtrar eventos onde há 'view' seguido por 'click' pelo mesmo usuário, no mesmo anúncio e na mesma campanha
df_filtered = df_sorted[
    ((df_sorted['action'] == 'view') & (df_sorted['next_action'] == 'click')) |
    ((df_sorted['action'] == 'click') & (df_sorted['prev_action'] == 'view'))   # Essa condição exclui da filtragem as 'next_action' = 'NaN'
    ]                                                                           # quando há transição de um ID de usuário pra outro

df_filtered

,timestamp,user_id,action,adId,campaignId,next_action,prev_action
183447,2016-04-03 13:12:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,click
156275,2016-04-03 20:50:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,view,view
163080,2016-07-21 19:04:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,view
224279,2016-09-02 19:26:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,view
138320,2016-12-01 19:36:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,click
...,...,...,...,...,...,...,...
135557,2016-09-17 21:13:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01,click,view
89742,2016-11-14 14:43:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,view,adId_10,campaignId_01,click,click
22729,2016-12-15 18:09:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,click,adId_10,campaignId_01,click,view
31330,2016-12-15 19:05:00,fff3f92a-9605-4db3-b485-640f74bf2ac3,view,adId_10,campaignId_01,click,click


In [ ]:
# Calculando a diferença de tempo entre os eventos onde 'view' é seguido de 'click'
df_filtered['time_between_events'] = df_filtered['timestamp'].shift(-1) - df_filtered['timestamp']
df_filtered.head(10)

<ipython-input-23-8e224593ac83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['time_between_events'] = df_filtered['timestamp'].shift(-1) - df_filtered['timestamp']


,timestamp,user_id,action,adId,campaignId,next_action,prev_action,time_between_events
183447,2016-04-03 13:12:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,click,0 days 07:38:00
156275,2016-04-03 20:50:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,view,view,108 days 22:14:00
163080,2016-07-21 19:04:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,view,43 days 00:22:00
224279,2016-09-02 19:26:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,view,90 days 00:10:00
138320,2016-12-01 19:36:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,click,0 days 01:06:00
150093,2016-12-01 20:42:00,00023420-6ead-4633-ba35-b1e0c5abe269,click,adId_01,campaignId_01,click,view,-297 days +13:15:00
226315,2016-02-09 09:57:00,000f0200-0918-4148-866c-341bd6eec354,view,adId_07,campaignId_03,click,NaN,52 days 00:19:00
173945,2016-04-01 10:16:00,000f0200-0918-4148-866c-341bd6eec354,click,adId_07,campaignId_03,click,view,172 days 10:58:00
193549,2016-09-20 21:14:00,000f0200-0918-4148-866c-341bd6eec354,view,adId_07,campaignId_03,click,click,27 days 11:40:00
191156,2016-10-18 08:54:00,000f0200-0918-4148-866c-341bd6eec354,click,adId_07,campaignId_03,click,view,-221 days +18:59:00


**OBS:**

Note que quando há transição de ID de um usuário para outro, um valor negativo de tempo é armazenado na coluna '*time_between_events*'. Isso ocorre porque o cálculo do tempo pega o '*timestamp*' da primeira linha de determinado novo ID de usuário (`df_filtered['timestamp'].shift(-1)`, um tempo anterior) e subtrai com o '*timestamp*' do ID do usuário anterior ('`df_filtered['timestamp']`', que é um tempo posterior).

Por exemplo, na linha 226315, o '*timestamp*' do novo usuário '*000f0200-0918-4148-866c-341bd6eec354*' é '*2016-02-09 09:57:00*' e na linha 150093, o usuário anterior '*00023420-6ead-4633-ba35-b1e0c5abe269*' possui '*timestamp*' de '*2016-12-01 20:42:00*'. Subtrair o segundo valor (tempo posterior) do primeiro valor (tempo anterior), o torna negativo. É importante observar que o tempo está ordenado localmente pra cada grupo de linhas relacionada a um único usuário.

Note também que o único tempo que importa e que será levado em consideração no cálculo dos tempos médios é o primeiro valor de '*time_between_events*' da dupla de linhas de ações do usuário para cada conjunto de campanhas e anúncios. Por isso, iremos desconsiderar e filtrar as linhas ímpares (index 1, 3, 5, ...) do conjunto de dados e preservar as linhas pares (0, 2, 4, ...) na célula abaixo.

In [ ]:
# Adicionar uma coluna para indicar se o índice é ímpar
df_filtered['is_odd'] = df_filtered.groupby(['user_id', 'adId', 'campaignId']).cumcount() % 2 == 0

# Filtrar apenas os valores ímpares
df_odd = df_filtered[df_filtered['is_odd']]
df_odd.head(10)

<ipython-input-25-7bc31c8d073c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['is_odd'] = df_filtered.groupby(['user_id', 'adId', 'campaignId']).cumcount() % 2 == 0


,timestamp,user_id,action,adId,campaignId,next_action,prev_action,time_between_events,is_odd
183447,2016-04-03 13:12:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,click,0 days 07:38:00,True
163080,2016-07-21 19:04:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,view,43 days 00:22:00,True
138320,2016-12-01 19:36:00,00023420-6ead-4633-ba35-b1e0c5abe269,view,adId_01,campaignId_01,click,click,0 days 01:06:00,True
226315,2016-02-09 09:57:00,000f0200-0918-4148-866c-341bd6eec354,view,adId_07,campaignId_03,click,NaN,52 days 00:19:00,True
193549,2016-09-20 21:14:00,000f0200-0918-4148-866c-341bd6eec354,view,adId_07,campaignId_03,click,click,27 days 11:40:00,True
248971,2016-03-12 03:53:00,00130041-b283-4154-8406-b4079dfe7c88,view,adId_08,campaignId_02,click,click,0 days 07:31:00,True
157309,2016-07-11 21:05:00,00130041-b283-4154-8406-b4079dfe7c88,view,adId_08,campaignId_02,click,click,0 days 09:46:00,True
171917,2016-08-12 14:41:00,00130041-b283-4154-8406-b4079dfe7c88,view,adId_08,campaignId_02,click,click,9 days 19:54:00,True
231806,2016-09-26 18:03:00,00130041-b283-4154-8406-b4079dfe7c88,view,adId_08,campaignId_02,click,click,1 days 06:30:00,True
235230,2016-11-25 08:14:00,00130041-b283-4154-8406-b4079dfe7c88,view,adId_08,campaignId_02,click,view,2 days 10:00:00,True


In [ ]:
# Agrupa por usuário, anúncio e campanha e calcular a média do tempo entre eventos
grouped_data = df_odd.groupby(['user_id', 'adId', 'campaignId']).agg({'time_between_events': 'mean'}).reset_index()
grouped_data

,user_id,adId,campaignId,time_between_events
0,00023420-6ead-4633-ba35-b1e0c5abe269,adId_01,campaignId_01,14 days 11:02:00
1,000f0200-0918-4148-866c-341bd6eec354,adId_07,campaignId_03,39 days 17:59:30
2,00130041-b283-4154-8406-b4079dfe7c88,adId_08,campaignId_02,2 days 10:00:00
3,00317295-4aa4-46ef-a849-26ade50bf3e7,adId_05,campaignId_02,3 days 16:01:24
4,0031aa2d-5988-4024-95d4-42ae50c09955,adId_03,campaignId_03,3 days 03:14:00
...,...,...,...,...
10362,ffe0582f-44ef-451d-81a6-b9ac3d3d4fa5,adId_07,campaignId_01,5 days 23:07:20
10363,ffe4204d-72b7-400a-b66e-c6171e00062e,adId_03,campaignId_01,6 days 18:53:48
10364,fff18213-eeaf-41de-91ff-79e04577df67,adId_08,campaignId_01,13 days 11:32:00
10365,fff3e8a9-b2d0-4b3c-892e-4bb224feb970,adId_05,campaignId_02,14 days 21:58:10


In [ ]:
# Ordena os grupos pela média do tempo de maneira ascendente
sorted_data = grouped_data.sort_values(by='time_between_events')

# Seleciona os 5 primeiros grupos (menores tempo)
top_5_pairs = sorted_data.head(5)

top_5_pairs

,user_id,adId,campaignId,time_between_events
5288,848eb13c-6881-45a3-bd62-05422be20cad,adId_05,campaignId_02,0 days 00:08:00
9912,f4db1c5e-7780-4beb-a36f-1322743d99d5,adId_05,campaignId_02,0 days 00:11:00
9407,e929e33e-791a-4b73-8a1b-63cc65238ecc,adId_10,campaignId_03,0 days 00:15:00
1604,28c5def5-3ede-4ead-ac85-13c1819ead4b,adId_01,campaignId_01,0 days 00:16:00
7963,c67d4a61-a911-4982-a685-33664bdcbd76,adId_06,campaignId_02,0 days 00:19:00


**Resposta:**

1. Ordenamos por '*user_id*' e '*timestamp*';
2. Criamos duas colunas de apoio para filtragem ([*'next_action', 'prev_action'*]);
3. Filtramos os dados para selecionar os pares de linhas de ações do usuário;
4. Calculamos os tempos entre pares de ações de '*view*' e '*click* e filtramos as linhas pares do conjunto de dados;
5. Reagrupamos por '*user_id*', '*adId*' e '*campaignId*' e calculamos as médias dos tempos médios contidos em '*time_between_events*' pra cada grupo de usuários, anuncio e campanha;
6. Ordenamos e selecionamos as 5 primeiras linhas, que são os pares de anúncio e campanha com menores médias de tempo entre dois eventos.

Logo, os pares de anúncio e campanha foram '*adld_05-campaignId_02*' duas vezes por usuários diferentes, '*adld_10-campaignId_03*', '*adld_01-campaignId_01*' e '*adld_06-campaignId_02*'.